In [1]:
from Tensor import Tensor
from nn import MLP
from optim import Adam
from typing import List

In [2]:
X = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

Y = [1.0, -1.0, -1.0, 1.0]

In [3]:
model = MLP(3, 2, 1)

In [4]:
def criterion(y_hats: List[Tensor], Y: List[float]) -> Tensor:
  return sum([ (y_hat-y)**2 for y_hat, y in zip(y_hats, Y)])

In [5]:
optim = Adam(model.parameters()) 

In [7]:
n_epochs = 1000

for i in range(n_epochs):
    y_hats = [model(x) for x in X]
    loss = criterion(y_hats, Y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    print(f'epoch: {i+1}, loss: {loss}')

epoch: 1, loss: Tensor(value=2.0992907852459202e-05)
epoch: 2, loss: Tensor(value=2.061064546550224e-05)
epoch: 3, loss: Tensor(value=2.0235004814374104e-05)
epoch: 4, loss: Tensor(value=1.9865877641627416e-05)
epoch: 5, loss: Tensor(value=1.950315734103897e-05)
epoch: 6, loss: Tensor(value=1.9146738934521606e-05)
epoch: 7, loss: Tensor(value=1.8796519049357607e-05)
epoch: 8, loss: Tensor(value=1.845239589567754e-05)
epoch: 9, loss: Tensor(value=1.8114269244255014e-05)
epoch: 10, loss: Tensor(value=1.778204040454106e-05)
epoch: 11, loss: Tensor(value=1.7455612203014386e-05)
epoch: 12, loss: Tensor(value=1.713488896176343e-05)
epoch: 13, loss: Tensor(value=1.681977647736344e-05)
epoch: 14, loss: Tensor(value=1.6510182000009172e-05)
epoch: 15, loss: Tensor(value=1.620601421290985e-05)
epoch: 16, loss: Tensor(value=1.5907183211946694e-05)
epoch: 17, loss: Tensor(value=1.561360048559641e-05)
epoch: 18, loss: Tensor(value=1.5325178895088406e-05)
epoch: 19, loss: Tensor(value=1.5041832654827

In [9]:
model.parameters()

[Tensor(value=0.36304536773644885),
 Tensor(value=-0.0825083573729334),
 Tensor(value=1.56050749056135),
 Tensor(value=0.36239939551972805),
 Tensor(value=0.23401891331110447),
 Tensor(value=0.08708718722959788),
 Tensor(value=-0.3835956948725126),
 Tensor(value=0.11929813251212942),
 Tensor(value=-0.5937451346457675),
 Tensor(value=0.2880462629775126),
 Tensor(value=0.21784653166705423)]